# Exercise 3: Topic-modeling with unsupervised learning

In this exercise we will approach **topic modeling** as an unsupervised learning problem. The first issue, though, is to have a working definition of what we mean by a *topic*.

A common choice is to model topics as *groups of words that co-occur* in documents about certain topics, e.g., "dog", "bone", "leash" is expected to occur more frequently in documents *about dogs*. This reduces topic modeling to finding these groups of co-occuring words and models documents as collections of topics in this regard. 

This fundamental assumption is going to guide our approach to trying to detect these co-occurence patterns in a toy dataset of news documents. Similar to the exercises about supervised learning we are going to use our knowledge of the data and domain to make assumptions that are translated into increasingly refined *inductive biases* and *data representations*.  

## Setup: Imports and Configuration

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn for traditional ML
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
import hdbscan

# Dimensionality reduction
import umap

# Sentence transformers for modern embeddings
from sentence_transformers import SentenceTransformer

# Visualization settings
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")
print("\nNote: If you get import errors, install missing packages:")
print("  pip install sentence-transformers scikit-learn pandas matplotlib seaborn hdbscan umap-learn")

### Pre-download Embedding Model

We'll download the sentence embedding model now so it's ready for Part 5. This is a small (~80MB), fast model that is trained as a language model to capture semantic meaning (similar to the one we trained in Exercise 2).

In [ ]:
# Pre-download the embedding model (takes ~30 seconds first time)
print("Downloading sentence embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Model ready!")
print(f"  Model: all-MiniLM-L6-v2")
print(f"  Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")
print(f"  Note: Larger models like 'all-mpnet-base-v2' give better quality but are slower")

## Preliminaries: Load and Explore the Dataset

We'll use the **BBC News dataset**: 2,225 news articles from the BBC across 5 categories (business, entertainment, politics, sport, tech). This is a clean, well-structured dataset that's fairly easy to work with.

**For the adventurous**: The code below works with any dataset having `text` and `category` columns. You can substitute:
- **20 Newsgroups**: International online discussions (use `sklearn.datasets.fetch_20newsgroups`)
- **Reuters**: International financial news (https://www.kaggle.com/datasets/nltkdata/reuters)
- **Your own data**: Any CSV with a text column! (if you don't have categories you can just skip the parts using them)

Now we're going to:
1. Load the data
2. Get some stats for the dataset to understand it better
3. Show the distribution of documents across categories 
4. DIsplay a few example documents

In [ ]:
# Load BBC News dataset
# Download from: https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category
# Or use the provided CSV file

# Option 1: Load from CSV file
try:
    df = pd.read_csv('../data/bbc-text.csv')
    print("✓ Loaded BBC News from local file")
    # Basic info
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(f"{df.head()}")
except FileNotFoundError:
    print("BBC News CSV not found. Please download it from https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category and place it in the '../data/' directory.")


In [ ]:
# Display dataset statistics
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)

# Number of documents
n_docs = len(df)
print(f"Total documents: {n_docs:,}")

# Category distribution
print(f"\nCategories ({df['category'].nunique()}):")
category_counts = df['category'].value_counts()
for cat, count in category_counts.items():
    print(f"  {cat}: {count} ({100*count/n_docs:.1f}%)")

# Document length statistics
df['doc_length'] = df['text'].str.split().str.len()
print(f"\nDocument length (words):")
print(f"  Mean: {df['doc_length'].mean():.0f}")
print(f"  Median: {df['doc_length'].median():.0f}")
print(f"  Min: {df['doc_length'].min()}")
print(f"  Max: {df['doc_length'].max()}")

# Vocabulary size (rough estimate)
all_words = ' '.join(df['text']).lower().split()
vocab_size = len(set(all_words))
print(f"\nApproximate vocabulary size: {vocab_size:,} unique words")
print(f"Total words in corpus: {len(all_words):,}")

print("=" * 60)

In [ ]:
# Visualize category distribution
plt.figure(figsize=(10, 5))
category_counts.plot(kind='bar', color='steelblue')
plt.title('Document Distribution Across Categories', fontsize=14, fontweight='bold')
plt.xlabel('Category', fontsize=12)
plt.ylabel('Number of Documents', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Show sample documents from different categories
print("=" * 80)
print("SAMPLE DOCUMENTS")
print("=" * 80)

for category in df['category'].unique()[:5]:  # Show up to 5 categories
    sample = df[df['category'] == category].iloc[0]
    print(f"\n{'='*80}")
    print(f"Category: {category.upper()}")
    print(f"{'-'*80}")
    # Show first 500 characters
    text_preview = sample['text'][:500]
    if len(sample['text']) > 500:
        text_preview += "..."
    print(text_preview)

print(f"\n{'='*80}")

### Diagnostic: Word Frequency Analysis

Since our modeling assumptions center relative word frequency and co-occurence, it makes sense to start with understanding what words dominate our corpus. This will reveal our first challenge.

In [ ]:
# Count word frequencies across entire corpus
all_words_lower = ' '.join(df['text']).lower().split()
word_freq = Counter(all_words_lower)
most_common = word_freq.most_common(30)

# Visualize
words, counts = zip(*most_common)

plt.figure(figsize=(14, 6))
plt.bar(range(len(words)), counts, color='coral')
plt.xticks(range(len(words)), words, rotation=45, ha='right')
plt.title('Top 30 Most Frequent Words in Corpus', fontsize=14, fontweight='bold')
plt.xlabel('Word', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nOBSERVATION: The most common words are 'the', 'to', 'a', 'of', 'and', etc.")
print("These appear in every document regardless of topic. (Often called 'stopwords' because they are uninformative and should probably be removed.)")
print("\nWe need to find what makes documents DIFFERENT, not what they share.")

---

## Iteration 1 - Baseline with Bag-of-Words + K-Means
We start with a simple representation: we translate each document into a vector that keeps track of the number of times each word occurs in the document. This removes all information about word-order and bundles everything together.

For the clustering algorithm we start with **k-means** that tries to find a predefined number (e.g. 5) *central points* (called centroids) in the data and then assigns each document to it's nearest centroid.

This will serve as our baseline.

**What we expect**: Poor results because we're counting all words equally, including uninformative ones, and because we assign each document a single "topic".

In [ ]:
# Create Bag-of-Words representation
print("Creating Bag-of-Words representation...")

# CountVectorizer converts text to word count matrix
# We deliberately KEEP stopwords to see the problem
vectorizer_bow = CountVectorizer(
    lowercase=True,
    min_df=2,
    max_features=5000,
    #stop_words='english'  # Used later
    stop_words=None # Note: NO stop_words parameter - we keep everything!
)

# Fit and transform documents
X_bow = vectorizer_bow.fit_transform(df['text'])

print(f"✓ Created document-term matrix")
print(f"  Shape: {X_bow.shape} (documents × vocabulary)")
print(f"  Vocabulary size: {len(vectorizer_bow.get_feature_names_out())}")
print(f"  Sparsity: {100 * (1 - X_bow.nnz / (X_bow.shape[0] * X_bow.shape[1])):.1f}% of entries are zero")
print(f"\n  Note: Each document is now a vector of {X_bow.shape[1]} word counts")

In [ ]:
# Apply K-Means clustering
print("Applying K-Means clustering...")

n_clusters = 5  # We know there are 5 categories in BBC News

kmeans_bow = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10,  # Run 10 times with different initializations, keep best
    max_iter=300
)

# Fit and predict cluster labels
clusters_bow = kmeans_bow.fit_predict(X_bow)

print(f"✓ K-Means clustering complete")
print(f"  Number of clusters: {n_clusters}")
print(f"  Iterations to converge: {kmeans_bow.n_iter_}")

# Add cluster labels to dataframe
df['cluster_bow'] = clusters_bow

# Show cluster sizes
print(f"\n  Cluster sizes:")
for i in range(n_clusters):
    count = (clusters_bow == i).sum()
    print(f"    Cluster {i}: {count} documents ({100*count/len(df):.1f}%)")

Did this give us a nice clustering? How can we tell? 

**Let's investigate!**

### Diagnostic 1: Top Words Per Cluster

Now we'll try to gauge the quality of our clustering through a series of diagnostics that will tell us different important characteristics of our clustering.

First let's see what words characterize each cluster. This will reveal a major problem with our baseline approach.

In [ ]:
# Extract top words for each cluster
def get_top_words_per_cluster(X, vectorizer, cluster_labels, n_words=10):
    """
    Find the most important words for each cluster by averaging
    word counts across all documents in the cluster.
    """
    feature_names = vectorizer.get_feature_names_out()
    n_clusters = len(np.unique(cluster_labels))
    
    top_words = {}
    
    for cluster_id in range(n_clusters):
        # Get all documents in this cluster
        cluster_mask = cluster_labels == cluster_id
        cluster_docs = X[cluster_mask]
        
        # Average word counts across cluster
        avg_counts = np.asarray(cluster_docs.mean(axis=0)).flatten()
        
        # Get top words
        top_indices = avg_counts.argsort()[-n_words:][::-1]
        top_words[cluster_id] = [(feature_names[i], avg_counts[i]) for i in top_indices]
    
    return top_words

# Get top words
top_words_bow = get_top_words_per_cluster(X_bow, vectorizer_bow, clusters_bow, n_words=10)

# Display
print("=" * 80)
print("TOP 10 WORDS PER CLUSTER (Bag-of-Words + K-Means)")
print("=" * 80)

for cluster_id, words in top_words_bow.items():
    print(f"\nCluster {cluster_id}:")
    word_list = [f"{word} ({count:.2f})" for word, count in words]
    print(f"  {', '.join(word_list)}")

print("\n" + "=" * 80)
print("PROBLEM: Top words are stopwords ('the', 'to', 'a', 'of')!")
print("These appear everywhere and tell us nothing about topics.")
print("\nWe need better weighting that downweights common words.")
print("=" * 80)

In [ ]:
# Visualize top words for each cluster
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for cluster_id in range(n_clusters):
    words, counts = zip(*top_words_bow[cluster_id])
    
    axes[cluster_id].barh(range(len(words)), counts, color='skyblue')
    axes[cluster_id].set_yticks(range(len(words)))
    axes[cluster_id].set_yticklabels(words)
    axes[cluster_id].set_xlabel('Average Count', fontsize=10)
    axes[cluster_id].set_title(f'Cluster {cluster_id} Top Words', fontsize=11, fontweight='bold')
    axes[cluster_id].invert_yaxis()
    axes[cluster_id].grid(axis='x', alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('Top Words Per Cluster (Bag-of-Words) - Dominated by Stopwords', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

Note how there is a huge overlap in the most frequent words across clusters and what seems to separate them is the absolute number of these words in each text - we accidentally clustered based on text-length, not topic!

### Diagnostic 2: Cluster Quality Metrics

We'll evaluate clusters using several metrics:

1. **Cluster balance**: Are clusters roughly equal in size, or is everything in one cluster?
2. **Cluster purity**: If we know true categories, how homogeneous are clusters?
3. **Top word distinctiveness**: Do clusters have unique characteristic words (do they represent distinct topics)?

These metrics tell us if clusters are meaningful, not just geometrically separated.

In [ ]:
# Calculate cluster quality metrics
print("Calculating cluster quality metrics...")

# 1. Cluster balance (entropy of size distribution)
from scipy.stats import entropy

cluster_sizes = np.bincount(clusters_bow)
cluster_balance = entropy(cluster_sizes / cluster_sizes.sum()) / np.log(n_clusters)

# 2. Cluster purity (if we have true labels)
def cluster_purity(labels_true, labels_pred):
    """Calculate purity: fraction of dominant class in each cluster"""
    contingency = pd.crosstab(labels_pred, labels_true)
    return (contingency.max(axis=1).sum() / len(labels_true))

purity_bow = cluster_purity(df['category'], clusters_bow)

# 3. Top word distinctiveness (how unique are top words?)
def top_word_overlap(top_words_dict, n_words=10):
    """Calculate average overlap between cluster top words"""
    all_top_words = [set([w for w, _ in words[:n_words]]) for words in top_words_dict.values()]
    overlaps = []
    for i in range(len(all_top_words)):
        for j in range(i+1, len(all_top_words)):
            overlap = len(all_top_words[i] & all_top_words[j]) / n_words
            overlaps.append(overlap)
    return np.mean(overlaps)

word_overlap_bow = top_word_overlap(top_words_bow)

print(f"✓ Cluster quality analysis complete")
print(f"\n  Cluster balance: {cluster_balance:.3f} (1.0 = perfectly balanced)")
print(f"  Cluster purity: {purity_bow:.3f} (1.0 = perfect match with categories)")
print(f"  Top word overlap: {word_overlap_bow:.3f} (0.0 = completely distinct)")
print(f"\n  Interpretation:")
if cluster_balance < 0.9:
    print(f"    ⚠️ Unbalanced: Some clusters dominate")
else:
    print(f"    ✅ Balanced: Clusters are fairly even in size")
if purity_bow < 0.5:
    print(f"    ⚠️ Low purity: Clusters don't match known categories well")
else:
    print(f"    ✅ High purity: Clusters match known categories well")
if word_overlap_bow > 0.3:
    print(f"    ⚠️ High overlap: Clusters share many top words (not distinctive)")
else:
    print(f"    ✅ Low overlap: Clusters are lexically distinct")

**Before moving on**: Try to go back and add stop_words='english' to the CountVectorizer to remove common words and improve clustering.

We'll try to visualize the size distribution and purity to get a feel for the clusters:

In [ ]:
# Visualize cluster balance and purity
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cluster size distribution
cluster_sizes_pct = cluster_sizes / cluster_sizes.sum() * 100
ax1.bar(range(n_clusters), cluster_sizes_pct, color='steelblue', edgecolor='black')
ax1.axhline(y=100/n_clusters, color='red', linestyle='--', linewidth=2, 
            label=f'Perfect balance ({100/n_clusters:.1f}%)')
ax1.set_xlabel('Cluster ID', fontsize=12)
ax1.set_ylabel('Percentage of Documents', fontsize=12)
ax1.set_title(f'Cluster Size Distribution (Balance: {cluster_balance:.3f})', 
              fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Cluster composition (true categories)
contingency = pd.crosstab(clusters_bow, df['category'], normalize='index') * 100
contingency.plot(kind='bar', stacked=True, ax=ax2, colormap='Set3', legend=False)
ax2.set_xlabel('Cluster ID', fontsize=12)
ax2.set_ylabel('Percentage', fontsize=12)
ax2.set_title(f'Cluster Composition by Category (Purity: {purity_bow:.3f})', 
              fontsize=12, fontweight='bold')
ax2.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nOBSERVATION: Clusters are unbalanced and have low purity.")
print("This confirms that BoW with stopwords doesn't create meaningful groupings.")

### Diagnostic 3: 2D Visualization with PCA

We can use **principle components analysis** to project our documents into a 2D space to see, if the clusters are geometrically separate. 

**Note**: We lose A LOT of information doing this, so be wary of reading too much information into this plot.

In [ ]:
# Reduce to 2D using PCA
print("Reducing to 2D with PCA...")
pca_bow = PCA(n_components=2, random_state=42)
X_bow_2d = pca_bow.fit_transform(X_bow.toarray())

print(f"✓ PCA complete")
print(f"  Variance explained: {100*pca_bow.explained_variance_ratio_.sum():.1f}%")
print(f"  (This is how much information we retained in 2D)")

In [ ]:
# Visualize clusters in 2D
plt.figure(figsize=(12, 8))

# Plot each cluster with different color
colors = plt.cm.Set2(np.linspace(0, 1, n_clusters))

for cluster_id in range(n_clusters):
    mask = clusters_bow == cluster_id
    plt.scatter(X_bow_2d[mask, 0], X_bow_2d[mask, 1], 
                c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Document Clusters in 2D (Bag-of-Words + K-Means)\nNo Distinct Cluster Boundaries', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nOBSERVATION: Clusters do not have distinct boundaries in 2D projection.")
print("Documents from different clusters are mixed together.")
print("\nThis confirms poor cluster separation due to uninformative features.")

### Reflection: What Did We Learn?

**Problems with Iteration 1**:
1. ❌ Top words are stopwords ("the", "to", "a")
2. ❌ Clusters are unbalanced (some very large, some very small)
3. ❌ Low purity (clusters don't match known categories)
4. ❌ High word overlap (clusters aren't distinctive) unless stopwords are removed.

**Why?** Bag-of-Words treats all words equally. Common words dominate, making all documents look similar. We risk accidentally clustering based on the absolute count of words rather than their relative frequency and co-occurence.

**Next step**: We'll start by improving our **representation** to *TF-IDF* which automatically downweights common words and upweights distinctive ones.

---

## Iteration 2 - Better Representation with TF-IDF + K-Means

**What we're changing**: ONLY the data representation (Bag-of-Words → TF-IDF)

**What stays the same**: K-Means algorithm, number of clusters (k=5), all other parameters

**What is TF-IDF?**

**TF-IDF** = Term Frequency × Inverse Document Frequency

- **TF (Term Frequency)**: How often does this word appear in this document?
- **IDF (Inverse Document Frequency)**: How rare is this word across all documents?

**The key insight**: 
- A word appearing in EVERY document (like "the") gets LOW weight
- A word appearing in only SPORTS articles (like "football") gets HIGH weight

This automatically downweights common words and upweights distinctive ones!

In [ ]:
# Create TF-IDF representation
print("Creating TF-IDF representation...")

# TfidfVectorizer: same as CountVectorizer but applies TF-IDF weighting
vectorizer_tfidf = TfidfVectorizer(
    lowercase=True,
    min_df=2,
    max_features=5000,
    stop_words='english' # If you want you can try without removing stopwords first
)

# Fit and transform
X_tfidf = vectorizer_tfidf.fit_transform(df['text'])

print(f"✓ Created TF-IDF matrix")
print(f"  Shape: {X_tfidf.shape} (same as before)")
print(f"  Vocabulary size: {len(vectorizer_tfidf.get_feature_names_out())}")
print(f"\n  Key difference: Values are now TF-IDF scores, not raw counts")
print(f"  Common words get low scores, rare distinctive words get high scores")

In [ ]:
# Apply K-Means clustering (SAME algorithm, different representation)
print("Applying K-Means clustering to TF-IDF vectors...")

kmeans_tfidf = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10,
    max_iter=300
)

clusters_tfidf = kmeans_tfidf.fit_predict(X_tfidf)

print(f"✓ K-Means clustering complete")
print(f"  Iterations to converge: {kmeans_tfidf.n_iter_}")

# Add to dataframe
df['cluster_tfidf'] = clusters_tfidf

# Show cluster sizes
print(f"\n  Cluster sizes:")
for i in range(n_clusters):
    count = (clusters_tfidf == i).sum()
    print(f"    Cluster {i}: {count} documents ({100*count/len(df):.1f}%)")

### Visualizing the TF-IDF Representation

Now let's see how TF-IDF changes the representation. The key difference: common words that occur at about similar rates across documents get downweighted, distinctive words that only occur in some documents get upweighted.

In [ ]:
# Visualize the SAME document with TF-IDF representation
sample_idx = 0
sample_vector_tfidf = X_tfidf[sample_idx].toarray().flatten()

# Get feature names
feature_names_tfidf = vectorizer_tfidf.get_feature_names_out()

# Find non-zero entries
nonzero_indices_tfidf = np.nonzero(sample_vector_tfidf)[0]
word_scores = [(feature_names_tfidf[i], sample_vector_tfidf[i]) for i in nonzero_indices_tfidf]
word_scores.sort(key=lambda x: x[1], reverse=True)

print("=" * 80)
print("SAME DOCUMENT (TF-IDF Representation)")
print("=" * 80)
print(f"\nOriginal text (first 300 characters):")
print(f"{df['text'][sample_idx][:300]}...")
print(f"\nDocument as TF-IDF vector:")
print(f"  Vector length: {len(sample_vector_tfidf)}")
print(f"  Non-zero entries: {len(nonzero_indices_tfidf)}")
print(f"\nTop 20 words by TF-IDF score:")
for word, score in word_scores[:20]:
    print(f"  '{word}': {score:.4f}")
print("\n✓ Notice: Top words seem topical for the article (without stopwords removed, notice how irrelevant words are no longer COMPELTELY dominant, but others are there as well).")
print("✓ Distinctive words have HIGH scores")
print("=" * 80)

In [ ]:
# Visualize TF-IDF scores for sample document
top_n = 30
words_tfidf, scores_tfidf = zip(*word_scores[:top_n])

plt.figure(figsize=(14, 6))
plt.barh(range(len(words_tfidf)), scores_tfidf, color='lightgreen')
plt.yticks(range(len(words_tfidf)), words_tfidf)
plt.xlabel('TF-IDF Score (higher = more distinctive)', fontsize=12)
plt.ylabel('Word', fontsize=12)
plt.title(f'TF-IDF: Top {top_n} Words in Sample Document\n✅ Distinctive words are emphasized!', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ IMPROVEMENT: Top words are now more topic-specific and meaningful!")
print("Generic words have been automatically downweighted (though it still makes sense to remove them to improve performance further).")

### Diagnostic 1: Top Words Per Cluster (TF-IDF)

Now let's see if the top words are more meaningful!

In [ ]:
# Get top words for TF-IDF clusters
top_words_tfidf = get_top_words_per_cluster(X_tfidf, vectorizer_tfidf, clusters_tfidf, n_words=10)

# Display
print("=" * 80)
print("TOP 10 WORDS PER CLUSTER (TF-IDF + K-Means)")
print("=" * 80)

for cluster_id, words in top_words_tfidf.items():
    print(f"\nCluster {cluster_id}:")
    word_list = [f"{word} ({score:.3f})" for word, score in words]
    print(f"  {', '.join(word_list)}")

print("\n" + "=" * 80)
print("✓ IMPROVEMENT: Top words are now meaningful and topic-specific!")
print("Notice words like 'film', 'game', 'government', 'technology', etc.")
print("These actually tell us what each cluster is about.")
print("=" * 80)

In [ ]:
# Visualize top words for each cluster
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for cluster_id in range(n_clusters):
    words, scores = zip(*top_words_tfidf[cluster_id])
    
    axes[cluster_id].barh(range(len(words)), scores, color='lightgreen')
    axes[cluster_id].set_yticks(range(len(words)))
    axes[cluster_id].set_yticklabels(words)
    axes[cluster_id].set_xlabel('Average TF-IDF Score', fontsize=10)
    axes[cluster_id].set_title(f'Cluster {cluster_id} Top Words', fontsize=11, fontweight='bold')
    axes[cluster_id].invert_yaxis()
    axes[cluster_id].grid(axis='x', alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('Top Words Per Cluster (TF-IDF) - Much More Meaningful!', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### Diagnostic 2: Cluster Quality Metrics (TF-IDF)

We'll analyse the quality with the same metrics as before:

In [ ]:
# Calculate cluster quality metrics for TF-IDF
print("Calculating cluster quality metrics for TF-IDF clustering...")

# Cluster balance (fixed formula)
cluster_sizes_tfidf = np.bincount(clusters_tfidf)
cluster_balance_tfidf = entropy(cluster_sizes_tfidf / cluster_sizes_tfidf.sum()) / np.log(n_clusters)

# Cluster purity
purity_tfidf = cluster_purity(df['category'], clusters_tfidf)

# Top word distinctiveness
word_overlap_tfidf = top_word_overlap(top_words_tfidf)

print(f"✓ Cluster quality analysis complete")
print(f"\n  Cluster balance: {cluster_balance_tfidf:.3f} (previous: {cluster_balance:.3f})")
print(f"  Cluster purity: {purity_tfidf:.3f} (previous: {purity_bow:.3f})")
print(f"  Top word overlap: {word_overlap_tfidf:.3f} (previous: {word_overlap_bow:.3f})")
print(f"\n  Improvements:")
print(f"    Balance: {cluster_balance_tfidf - cluster_balance:+.3f}")
print(f"    Purity: {purity_tfidf - purity_bow:+.3f}")
print(f"    Distinctiveness: {word_overlap_bow - word_overlap_tfidf:+.3f} (lower overlap is better)")

In [ ]:
# Compare cluster quality metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Row 1: Cluster size distributions
cluster_sizes_bow_pct = cluster_sizes / cluster_sizes.sum() * 100
cluster_sizes_tfidf_pct = cluster_sizes_tfidf / cluster_sizes_tfidf.sum() * 100

axes[0, 0].bar(range(n_clusters), cluster_sizes_bow_pct, color='steelblue', edgecolor='black')
axes[0, 0].axhline(y=100/n_clusters, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Cluster ID', fontsize=11)
axes[0, 0].set_ylabel('Percentage of Documents', fontsize=11)
axes[0, 0].set_title(f'BoW: Balance = {cluster_balance:.3f}', fontsize=12, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

axes[0, 1].bar(range(n_clusters), cluster_sizes_tfidf_pct, color='lightgreen', edgecolor='black')
axes[0, 1].axhline(y=100/n_clusters, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Cluster ID', fontsize=11)
axes[0, 1].set_ylabel('Percentage of Documents', fontsize=11)
axes[0, 1].set_title(f'TF-IDF: Balance = {cluster_balance_tfidf:.3f}', fontsize=12, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# Row 2: Cluster purity (if we have true labels)
contingency_bow = pd.crosstab(clusters_bow, df['category'], normalize='index') * 100
contingency_tfidf = pd.crosstab(clusters_tfidf, df['category'], normalize='index') * 100

contingency_bow.plot(kind='bar', stacked=True, ax=axes[1, 0], colormap='Set3', legend=False)
axes[1, 0].set_xlabel('Cluster ID', fontsize=11)
axes[1, 0].set_ylabel('Percentage', fontsize=11)
axes[1, 0].set_title(f'BoW: Purity = {purity_bow:.3f}', fontsize=12, fontweight='bold')
axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)

contingency_tfidf.plot(kind='bar', stacked=True, ax=axes[1, 1], colormap='Set3', legend=True)
axes[1, 1].set_xlabel('Cluster ID', fontsize=11)
axes[1, 1].set_ylabel('Percentage', fontsize=11)
axes[1, 1].set_title(f'TF-IDF: Purity = {purity_tfidf:.3f}', fontsize=12, fontweight='bold')
axes[1, 1].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=0)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.suptitle('Cluster Quality Comparison: TF-IDF Shows Clear Improvement', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Purity and balance both increase drastically with TF-IDF compared to BoW.")

### Diagnostic 3: 2D Visualization with PCA

Again we'll try to visualize the representation in 2d to see if the clusters are better separated.

In [ ]:
# Reduce to 2D using PCA
print("Reducing to 2D with PCA...")
pca_tfidf = PCA(n_components=2, random_state=42)
X_tfidf_2d = pca_tfidf.fit_transform(X_tfidf.toarray())

print(f"✓ PCA complete")
print(f"  Variance explained: {100*pca_tfidf.explained_variance_ratio_.sum():.1f}%")
print(f"  (This is how much information we retained in 2D)")

In [ ]:
# Visualize clusters in 2D
plt.figure(figsize=(12, 8))

# Plot each cluster with different color
colors = plt.cm.Set2(np.linspace(0, 1, n_clusters))

for cluster_id in range(n_clusters):
    mask = clusters_tfidf == cluster_id
    plt.scatter(X_tfidf_2d[mask, 0], X_tfidf_2d[mask, 1], 
                c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Document Clusters in 2D (TF-IDF + K-Means)\nCluster boundaries become clearer', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nOBSERVATION: Clusters are more distinct, although still with a large overlap in 2D projection.")
print("\nOur clustering seems to have improved.")

 ### Reflection: What Did We Learn?

**Improvements from Iteration 2**:
1. ✅ Top words are more meaningful and topic-specific
2. ✅ Clusters are more balanced (better size distribution)
3. ✅ Higher purity (clusters match known categories)
4. ✅ Lower word overlap (clusters are more distinctive)

**Remaining problems**:
1. ❌ K-Means forces hard assignments (one cluster per document)
2. ❌ Many documents discuss multiple topics
3. ❌ Real articles often span categories (e.g., "technology in politics")

**Next step**: Use LDA for soft clustering where documents can belong to multiple topics.

---

## Iteration 3 - Soft Clustering with LDA

**What we're changing**: ONLY the algorithm (K-Means → LDA)

**What stays the same**: Number of topics (k=5)

**Important note about representation**: We need to go back to count-based features (Bag-of-Words) for LDA, not TF-IDF!

**What is LDA?**

**LDA** = Latent Dirichlet Allocation (a probabilistic topic model)

**Key differences from K-Means**:
- **Soft clustering**: Each document is a MIXTURE of topics (e.g., 60% politics, 30% technology, 10% business)
- **Interpretable topics**: Each topic is a distribution over words
- **Generative model**: Assumes documents are created by mixing topics

**The key insight**: Real documents often discuss multiple themes. LDA captures this naturally!

**Why not TF-IDF?** LDA is a probabilistic model that assumes word counts follow a multinomial distribution. TF-IDF values are weighted scores, not counts, which violates LDA's mathematical assumptions. Using TF-IDF with LDA can lead to poor topic quality and convergence issues.

In [ ]:
# Create Bag-of-Words representation for LDA
print("Creating Bag-of-Words representation for LDA...")

# LDA requires count-based features, not TF-IDF weighted scores
# We'll use CountVectorizer with stopwords removed (building on Iteration 2's insight)
vectorizer_lda = CountVectorizer(
    lowercase=True,
    min_df=2,
    max_features=5000,
    stop_words='english'  # Remove stopwords based on Iteration 2 learning
)

# Fit and transform documents
X_lda = vectorizer_lda.fit_transform(df['text'])

print(f"✓ Created document-term matrix for LDA")
print(f"  Shape: {X_lda.shape} (documents × vocabulary)")
print(f"  Vocabulary size: {len(vectorizer_lda.get_feature_names_out())}")
print(f"  Sparsity: {100 * (1 - X_lda.nnz / (X_lda.shape[0] * X_lda.shape[1])):.1f}% of entries are zero")
print(f"\n  Note: Using raw counts (not TF-IDF) because LDA is a probabilistic model")
print(f"  But we keep the stopword removal insight from Iteration 2!")

In [ ]:
# Apply LDA topic modeling
print("Applying LDA topic modeling...")

n_topics = 5  # Same as number of clusters before

lda_model = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    max_iter=20,
    learning_method='batch',
    n_jobs=-1  # Use all CPU cores
)

# Fit model and get document-topic distributions
doc_topic_dist = lda_model.fit_transform(X_lda)

print(f"✓ LDA modeling complete")
print(f"  Number of topics: {n_topics}")
print(f"  Iterations: {lda_model.n_iter_}")
print(f"  Final perplexity: {lda_model.perplexity(X_lda):.2f} (lower is better)")

# Get dominant topic for each document (for comparison with K-Means)
dominant_topics = doc_topic_dist.argmax(axis=1)
df['topic_lda'] = dominant_topics

print(f"\n  Topic sizes (by dominant topic):")
for i in range(n_topics):
    count = (dominant_topics == i).sum()
    print(f"    Topic {i}: {count} documents ({100*count/len(df):.1f}%)")

### Diagnostic 1: Topic-Word Distributions

Let's see what words characterize each topic. Unlike K-Means, LDA gives us probability distributions over words.

In [ ]:
# Extract top words for each topic
def get_top_words_per_topic(model, vectorizer, n_words=10):
    """
    Get top words for each topic based on word probabilities.
    """
    feature_names = vectorizer.get_feature_names_out()
    top_words = {}
    
    for topic_id, topic_dist in enumerate(model.components_):
        # Get top word indices
        top_indices = topic_dist.argsort()[-n_words:][::-1]
        # Get words and their probabilities
        top_words[topic_id] = [(feature_names[i], topic_dist[i]) for i in top_indices]
    
    return top_words

# Get top words for each topic (using TF-IDF vocabulary)
top_words_lda = get_top_words_per_topic(lda_model, vectorizer_lda, n_words=10)

# Display
print("=" * 80)
print("TOP 10 WORDS PER TOPIC (LDA with Bag-of-Words)")
print("=" * 80)

for topic_id, words in top_words_lda.items():
    print(f"\nTopic {topic_id}:")
    word_list = [f"{word} ({prob:.3f})" for word, prob in words]
    print(f"  {', '.join(word_list)}")

print("\n" + "=" * 80)
print("✓ OBSERVATION: Topics are coherent and interpretable!")
print("Each topic has a clear theme (e.g., sports, politics, technology).")
print("=" * 80)

In [ ]:
# Visualize top words for each topic
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for topic_id in range(n_topics):
    words, scores = zip(*top_words_lda[topic_id])
    
    axes[topic_id].barh(range(len(words)), scores, color='mediumpurple')
    axes[topic_id].set_yticks(range(len(words)))
    axes[topic_id].set_yticklabels(words)
    axes[topic_id].set_xlabel('Probability in Topic', fontsize=10)
    axes[topic_id].set_title(f'Topic {topic_id} Top Words', fontsize=11, fontweight='bold')
    axes[topic_id].invert_yaxis()
    axes[topic_id].grid(axis='x', alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('Top Words Per Topic (LDA) - Coherent Themes!', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### Diagnostic 2: Cluster Quality Metrics (LDA)

In [ ]:
# Calculate cluster quality metrics for LDA
print("Calculating cluster quality metrics for LDA...")

# Cluster balance (using dominant topics)
topic_sizes = np.bincount(dominant_topics)
cluster_balance_lda = entropy(topic_sizes / topic_sizes.sum()) / np.log(n_topics)

# Cluster purity
purity_lda = cluster_purity(df['category'], dominant_topics)

# Top word distinctiveness
word_overlap_lda = top_word_overlap(top_words_lda)

print(f"✓ Cluster quality analysis complete")
print(f"\n  Cluster balance: {cluster_balance_lda:.3f}")
print(f"  Cluster purity: {purity_lda:.3f}")
print(f"  Top word overlap: {word_overlap_lda:.3f}")
print(f"\n  Comparison:")
print(f"    BoW:    balance={cluster_balance:.3f}, purity={purity_bow:.3f}")
print(f"    TF-IDF: balance={cluster_balance_tfidf:.3f}, purity={purity_tfidf:.3f}")
print(f"    LDA:    balance={cluster_balance_lda:.3f}, purity={purity_lda:.3f}")

In [ ]:
# Compare all methods
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
# Row 1: Cluster size distributions
methods = ['BoW\n+K-Means', 'TF-IDF\n+K-Means', 'BoW\n+LDA']
balances = [cluster_balance, cluster_balance_tfidf, cluster_balance_lda]
size_dists = [
    cluster_sizes / cluster_sizes.sum() * 100,
    cluster_sizes_tfidf / cluster_sizes_tfidf.sum() * 100,
    topic_sizes / topic_sizes.sum() * 100
]
colors_bar = ['steelblue', 'lightgreen', 'mediumpurple']

for i, (method, balance, sizes, color) in enumerate(zip(methods, balances, size_dists, colors_bar)):
    axes[0, i].bar(range(n_clusters), sizes, color=color, edgecolor='black')
    axes[0, i].axhline(y=100/n_clusters, color='red', linestyle='--', linewidth=2)
    axes[0, i].set_xlabel('Cluster/Topic ID', fontsize=9)
    axes[0, i].set_ylabel('Percentage', fontsize=9)
    axes[0, i].set_title(f'{method}\nBalance = {balance:.3f}', fontsize=10, fontweight='bold')
    axes[0, i].grid(axis='y', alpha=0.3)

# Row 2: Cluster purity (composition)
purities = [purity_bow, purity_tfidf, purity_lda]
cluster_labels_all = [clusters_bow, clusters_tfidf, dominant_topics]
contingencies = [
    pd.crosstab(labels, df['category'], normalize='index') * 100
    for labels in cluster_labels_all
]

for i, (method, purity, cont, color) in enumerate(zip(methods, purities, contingencies, colors_bar)):
    cont.plot(kind='bar', stacked=True, ax=axes[1, i], colormap='Set3', legend=(i==2))
    axes[1, i].set_xlabel('Cluster/Topic ID', fontsize=9)
    axes[1, i].set_ylabel('Percentage', fontsize=9)
    axes[1, i].set_title(f'{method}\nPurity = {purity:.3f}', fontsize=10, fontweight='bold')
    axes[1, i].set_xticklabels(axes[1, i].get_xticklabels(), rotation=0)
    axes[1, i].grid(axis='y', alpha=0.3)
    if i == 2:
        axes[1, i].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.suptitle('Progression so far', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Purity and balance both decreases from TF-IDF to LDA, but LDA provides a SOFTER clustering that might better reflects document complexity.")

### BONUS: Soft Clustering in Action

Skip this exercise if you lack time.

Let's see how LDA's soft clustering handles documents that span multiple topics. Unlike K-Means which forces a hard assignment, LDA gives us a probability distribution over topics for each document.

In [ ]:
# Find documents with ambiguous topic assignments
# These are documents where the dominant topic has low probability (< 0.5)
# or where multiple topics have similar probabilities

# Calculate topic entropy for each document (higher = more ambiguous)
from scipy.stats import entropy as scipy_entropy

doc_topic_entropy = np.array([scipy_entropy(dist) for dist in doc_topic_dist])

# Get max topic probability for each document
max_topic_prob = doc_topic_dist.max(axis=1)

# Find ambiguous documents (low max probability or high entropy)
ambiguous_mask = (max_topic_prob < 0.5) | (doc_topic_entropy > 1.0)
ambiguous_indices = np.where(ambiguous_mask)[0]

print(f"Found {len(ambiguous_indices)} ambiguous documents ({100*len(ambiguous_indices)/len(df):.1f}%)")
print(f"  Low max probability (< 0.5): {(max_topic_prob < 0.5).sum()}")
print(f"  High entropy (> 1.0): {(doc_topic_entropy > 1.0).sum()}")

In [ ]:
# Visualize topic distribution for a few ambiguous documents
print("=" * 80)
print("EXAMPLES OF SOFT CLUSTERING: AMBIGUOUS DOCUMENTS")
print("=" * 80)

# Show 3 most ambiguous documents
most_ambiguous_indices = doc_topic_entropy.argsort()[-3:][::-1]

for rank, idx in enumerate(most_ambiguous_indices, 1):
    print(f"\n{'='*80}")
    print(f"Example {rank}: Document {idx}")
    print(f"Category: {df.iloc[idx]['category']}")
    print(f"Topic entropy: {doc_topic_entropy[idx]:.3f} (higher = more mixed)")
    print(f"{'-'*80}")
    
    # Show topic distribution
    topic_dist = doc_topic_dist[idx]
    print("\nTopic distribution:")
    for topic_id in range(n_topics):
        prob = topic_dist[topic_id]
        if prob > 0.05:  # Only show topics with > 5% probability
            top_words = [w for w, _ in top_words_lda[topic_id][:5]]
            print(f"  Topic {topic_id}: {prob*100:5.1f}% ({', '.join(top_words)})")
    
    # Show text preview
    print(f"\nText preview:")
    print(f"{df.iloc[idx]['text'][:400]}...")
    
    # Compare with K-Means hard assignment
    kmeans_cluster = clusters_tfidf[idx]
    print(f"\nK-Means would assign this to: Cluster {kmeans_cluster} (hard assignment)")
    print(f"LDA shows it's a mixture of topics (soft assignment)")

print(f"\n{'='*80}")

In [ ]:
# Visualize the distribution of topic probabilities
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Distribution of max topic probabilities
axes[0].hist(max_topic_prob, bins=30, color='mediumpurple', edgecolor='black', alpha=0.7)
axes[0].axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Ambiguous threshold')
axes[0].set_xlabel('Maximum Topic Probability', fontsize=12)
axes[0].set_ylabel('Number of Documents', fontsize=12)
axes[0].set_title('Distribution of Topic Certainty\n(Lower = More Ambiguous)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Right: Distribution of topic entropy
axes[1].hist(doc_topic_entropy, bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[1].axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='High entropy threshold')
axes[1].set_xlabel('Topic Entropy', fontsize=12)
axes[1].set_ylabel('Number of Documents', fontsize=12)
axes[1].set_title('Distribution of Topic Mixing\n(Higher = More Mixed)', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.suptitle('Soft Clustering: Many Documents Span Multiple Topics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n✓ INSIGHT: {(max_topic_prob < 0.5).sum()} documents ({100*(max_topic_prob < 0.5).sum()/len(df):.1f}%) have max topic probability < 50%")
print("These documents might discuss multiple themes - soft clustering is able to capture this.")

In [ ]:
# Visualize clusters in 2D
plt.figure(figsize=(12, 8))

for cluster_id in range(n_clusters):
    mask = clusters_tfidf == cluster_id
    plt.scatter(X_tfidf_2d[mask, 0], X_tfidf_2d[mask, 1],
                c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Document Clusters in 2D (TF-IDF + K-Means)\nBetter Separation', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ IMPROVEMENT: Clusters are more clearly separated than with Bag-of-Words. (More of a sanity check than anything else)")

### Reflection: What Did We Learn?

**Problems with lexical representations (BoW/TF-IDF)**:
1. ❌ Ignore word order ("dog bites man" vs "man bites dog" look similar)
2. ❌ No semantic understanding (synonyms treated as different words)
3. ❌ Sparse, high-dimensional vectors (5000+ dimensions)
4. ❌ Can't capture context or nuance

**Improvements so far**:
1. ✅ TF-IDF better than BoW (downweights common words)
2. ✅ LDA provides soft clustering (documents can have multiple topics)
3. ✅ Stopword removal helps focus on content words

**Next step**: Move beyond lexical representations to **semantic embeddings** that capture meaning, not just word co-occurrence.

---

## Iteration 4 - Semantic Representations with Sentence Embeddings

**What we're changing**: ONLY the data representation (TF-IDF → Sentence Embeddings)

**What stays the same**: K-Means algorithm, number of clusters (k=5)

**What are Sentence Embeddings?**

**Sentence Embeddings** = Dense vector representations that capture semantic meaning

**Key differences from TF-IDF**:
- **Dense vs Sparse**: 384 dimensions (dense) vs 5000+ dimensions (sparse)
- **Semantic vs Lexical**: Captures meaning vs counts words
- **Word order matters**: "dog bites man" ≠ "man bites dog"
- **Context-aware**: Same word, different meanings in different contexts
- **Pre-trained**: Learned from massive text corpora (like the language model in Exercise 2)

**The key insight**: Documents with similar meanings should have similar embeddings, even if they use different words!

**Why K-Means (not LDA)?** LDA requires count data (multinomial distribution). Embeddings are continuous vectors, so K-Means is the standard choice.

In [ ]:
# Generate sentence embeddings for all documents
print("Generating sentence embeddings...")
print(f"  Using model: {embedding_model}")
print(f"  Processing {len(df)} documents...")

# Encode all documents (this may take a minute)
# The model was already loaded in the setup section
embeddings = embedding_model.encode(
    df['text'].tolist(),
    show_progress_bar=True,
    batch_size=32
)

print(f"\n✓ Embeddings generated!")
print(f"  Shape: {embeddings.shape}")
print(f"  Embedding dimension: {embeddings.shape[1]}")
print(f"  Much smaller than TF-IDF: {embeddings.shape[1]} vs {X_tfidf.shape[1]} dimensions")
print(f"  Dense representation: every dimension has a value (no zeros)")

### Visualizing Semantic Similarity

Let's verify that embeddings capture semantic meaning by looking at similar documents.

In [ ]:
# Find semantically similar documents
from sklearn.metrics.pairwise import cosine_similarity

# Pick a sample document
sample_idx = 0
sample_text = df.iloc[sample_idx]['text']
sample_category = df.iloc[sample_idx]['category']

print("=" * 80)
print("SEMANTIC SIMILARITY DEMONSTRATION")
print("=" * 80)
print(f"\nSample document (Category: {sample_category}):")
print(f"{sample_text[:300]}...")

# Calculate cosine similarity between sample and all other documents
sample_embedding = embeddings[sample_idx].reshape(1, -1)
similarities = cosine_similarity(sample_embedding, embeddings)[0]

# Get top 5 most similar documents (excluding the sample itself)
similar_indices = similarities.argsort()[::-1][1:6]

print(f"\n{'='*80}")
print("TOP 5 MOST SIMILAR DOCUMENTS:")
print(f"{'='*80}")

for rank, idx in enumerate(similar_indices, 1):
    similarity = similarities[idx]
    category = df.iloc[idx]['category']
    text = df.iloc[idx]['text']
    
    print(f"\n{rank}. Similarity: {similarity:.3f} | Category: {category}")
    print(f"   {text[:200]}...")

print(f"\n{'='*80}")
print("✓ OBSERVATION: Similar documents have high cosine similarity")
print("Notice how semantically related documents are found, even with different words!")
print("=" * 80)

In [ ]:
# Visualize embedding space in 2D using PCA
print("Visualizing embedding space in 2D...")

# Reduce embeddings to 2D for visualization
pca_emb = PCA(n_components=2, random_state=42)
embeddings_2d = pca_emb.fit_transform(embeddings)

print(f"✓ PCA complete")
print(f"  Variance explained: {100*pca_emb.explained_variance_ratio_.sum():.1f}%")

# Plot by category
plt.figure(figsize=(12, 8))

for category in df['category'].unique():
    mask = df['category'] == category
    plt.scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
                label=category, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('Document Embeddings in 2D Space\nDocuments cluster by semantic similarity', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ OBSERVATION: Documents with similar meanings cluster together")
print("Even in 2D, we can see semantic structure!")

In [ ]:
# Apply K-Means clustering on embeddings
print("Applying K-Means clustering to embeddings...")

kmeans_emb = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10,
    max_iter=300
)

clusters_emb = kmeans_emb.fit_predict(embeddings)

print(f"✓ K-Means clustering complete")
print(f"  Number of clusters: {n_clusters}")
print(f"  Iterations to converge: {kmeans_emb.n_iter_}")

# Add cluster labels to dataframe
df['cluster_emb'] = clusters_emb

# Show cluster sizes
print(f"\n  Cluster sizes:")
for i in range(n_clusters):
    count = (clusters_emb == i).sum()
    print(f"    Cluster {i}: {count} documents ({100*count/len(df):.1f}%)")

### Diagnostic 1: Top Words Per Cluster (Embeddings)

Since embeddings don't directly give us words, we'll reuse TF-IDF to look at the most common words in documents assigned to each cluster.

In [ ]:
# Get top words for embedding-based clusters
# We'll use TF-IDF scores within each cluster to find characteristic words
top_words_emb = get_top_words_per_cluster(X_tfidf, vectorizer_tfidf, clusters_emb, n_words=10)

# Display
print("=" * 80)
print("TOP 10 WORDS PER CLUSTER (Embeddings + K-Means)")
print("=" * 80)

for cluster_id, words in top_words_emb.items():
    print(f"\nCluster {cluster_id}:")
    word_list = [f"{word} ({score:.3f})" for word, score in words]
    print(f"  {', '.join(word_list)}")

print("\n" + "=" * 80)
print("✓ OBSERVATION: Clusters based on semantic meaning")
print("Words reflect the semantic themes, not just lexical co-occurrence")
print("=" * 80)

In [ ]:
# Visualize top words for each cluster
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for cluster_id in range(n_clusters):
    words, scores = zip(*top_words_emb[cluster_id])
    
    axes[cluster_id].barh(range(len(words)), scores, color='teal')
    axes[cluster_id].set_yticks(range(len(words)))
    axes[cluster_id].set_yticklabels(words)
    axes[cluster_id].set_xlabel('Average TF-IDF Score', fontsize=10)
    axes[cluster_id].set_title(f'Cluster {cluster_id} Top Words', fontsize=11, fontweight='bold')
    axes[cluster_id].invert_yaxis()
    axes[cluster_id].grid(axis='x', alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('Top Words Per Cluster (Embeddings + K-Means)', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### Diagnostic 2: Cluster Quality Metrics (Embeddings)

In [ ]:
# Calculate cluster quality metrics for embeddings
print("Calculating cluster quality metrics for embeddings...")

# Cluster balance
cluster_sizes_emb = np.bincount(clusters_emb)
cluster_balance_emb = entropy(cluster_sizes_emb / cluster_sizes_emb.sum()) / np.log(n_clusters)

# Cluster purity
purity_emb = cluster_purity(df['category'], clusters_emb)

# Top word distinctiveness
word_overlap_emb = top_word_overlap(top_words_emb)

print(f"✓ Cluster quality analysis complete")
print(f"\n  Cluster balance: {cluster_balance_emb:.3f}")
print(f"  Cluster purity: {purity_emb:.3f}")
print(f"  Top word overlap: {word_overlap_emb:.3f}")
print(f"\n  Comparison:")
print(f"    BoW:       balance={cluster_balance:.3f}, purity={purity_bow:.3f}")
print(f"    TF-IDF:    balance={cluster_balance_tfidf:.3f}, purity={purity_tfidf:.3f}")
print(f"    LDA:       balance={cluster_balance_lda:.3f}, purity={purity_lda:.3f}")
print(f"    Embeddings: balance={cluster_balance_emb:.3f}, purity={purity_emb:.3f}")

In [ ]:
# Compare all methods including embeddings
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Row 1: Cluster size distributions
methods = ['BoW\n+K-Means', 'TF-IDF\n+K-Means', 'BoW\n+LDA', 'Embeddings\n+K-Means']
balances = [cluster_balance, cluster_balance_tfidf, cluster_balance_lda, cluster_balance_emb]
size_dists = [
    cluster_sizes / cluster_sizes.sum() * 100,
    cluster_sizes_tfidf / cluster_sizes_tfidf.sum() * 100,
    topic_sizes / topic_sizes.sum() * 100,
    cluster_sizes_emb / cluster_sizes_emb.sum() * 100
]
colors_bar = ['steelblue', 'lightgreen', 'mediumpurple', 'teal']

for i, (method, balance, sizes, color) in enumerate(zip(methods, balances, size_dists, colors_bar)):
    axes[0, i].bar(range(n_clusters), sizes, color=color, edgecolor='black')
    axes[0, i].axhline(y=100/n_clusters, color='red', linestyle='--', linewidth=2)
    axes[0, i].set_xlabel('Cluster/Topic ID', fontsize=9)
    axes[0, i].set_ylabel('Percentage', fontsize=9)
    axes[0, i].set_title(f'{method}\nBalance = {balance:.3f}', fontsize=10, fontweight='bold')
    axes[0, i].grid(axis='y', alpha=0.3)

# Row 2: Cluster purity (composition)
purities = [purity_bow, purity_tfidf, purity_lda, purity_emb]
cluster_labels_all = [clusters_bow, clusters_tfidf, dominant_topics, clusters_emb]
contingencies = [
    pd.crosstab(labels, df['category'], normalize='index') * 100
    for labels in cluster_labels_all
]

for i, (method, purity, cont, color) in enumerate(zip(methods, purities, contingencies, colors_bar)):
    cont.plot(kind='bar', stacked=True, ax=axes[1, i], colormap='Set3', legend=(i==3))
    axes[1, i].set_xlabel('Cluster/Topic ID', fontsize=9)
    axes[1, i].set_ylabel('Percentage', fontsize=9)
    axes[1, i].set_title(f'{method}\nPurity = {purity:.3f}', fontsize=10, fontweight='bold')
    axes[1, i].set_xticklabels(axes[1, i].get_xticklabels(), rotation=0)
    axes[1, i].grid(axis='y', alpha=0.3)
    if i == 3:
        axes[1, i].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.suptitle('Progression: Lexical → Semantic Representations', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✓ IMPROVEMENT: Embeddings achieve even better purity and balance. We've almost constructued the original categories from topics alone!")
print("Semantic understanding helps group documents more meaningfully!")

### Diagnostic 3: 2D Visualization Comparison

In [ ]:
# Compare TF-IDF vs Embeddings clustering in 2D
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left: TF-IDF + K-Means
for cluster_id in range(n_clusters):
    mask = clusters_tfidf == cluster_id
    ax1.scatter(X_tfidf_2d[mask, 0], X_tfidf_2d[mask, 1],
                c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

ax1.set_xlabel('First Principal Component', fontsize=12)
ax1.set_ylabel('Second Principal Component', fontsize=12)
ax1.set_title(f'TF-IDF + K-Means (Purity: {purity_tfidf:.3f})', fontsize=12, fontweight='bold')
ax1.legend(fontsize=9, loc='best')
ax1.grid(alpha=0.3)

# Right: Embeddings + K-Means
for cluster_id in range(n_clusters):
    mask = clusters_emb == cluster_id
    ax2.scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
                c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

ax2.set_xlabel('First Principal Component', fontsize=12)
ax2.set_ylabel('Second Principal Component', fontsize=12)
ax2.set_title(f'Embeddings + K-Means (Purity: {purity_emb:.3f})', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9, loc='best')
ax2.grid(alpha=0.3)

plt.suptitle('Lexical vs Semantic Clustering', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n✓ OBSERVATION: Embeddings create more semantically coherent clusters")

### Reflection: What Did We Learn?

**Improvements with Embeddings**:
1. ✅ **Semantic understanding**: Captures meaning, not just word counts
2. ✅ **Word order matters**: "dog bites man" ≠ "man bites dog"
3. ✅ **Dense representation**: 384 dimensions vs 5000+ (more efficient)
4. ✅ **Better clustering**: Higher purity, balance and low top-word overlap
5. ✅ **Synonym handling**: Similar meanings → similar vectors

**Trade-offs**:
- ⚠️ **Computational cost**: Generating embeddings takes time
- ⚠️ **Black box**: Harder to interpret than word counts
- ⚠️ **Model dependency**: Quality depends on pre-training data

**When to use embeddings**:
- ✅ Semantic similarity is important
- ✅ Documents use varied vocabulary
- ✅ You have computational resources
- ✅ You want state-of-the-art performance

**Next step**: Combine embeddings with density-based clustering (HDBSCAN) to automatically discover the number of topics!

---

## EXTRA CREDIT: Iteration 5 - Automatic Cluster Discovery with HDBSCAN

**What we're changing**: Algorithm (K-Means → HDBSCAN) + addressing dimensionality

**What stays the same**: Semantic embeddings representation

**The problem with k=5**: We've been assuming 5 topics because the dataset has 5 categories. But:
- Are news categories the same as topics?
- Topics might be more granular (e.g., "elections", "stock markets", "football transfers")
- Or more abstract (e.g., "conflict", "innovation", "regulation")

**What is HDBSCAN?**

**HDBSCAN** = Hierarchical Density-Based Spatial Clustering of Applications with Noise

**Key features**:
- **No need to specify k**: Discovers clusters based on data density
- **Handles noise**: Can label outliers (documents that don't fit any cluster)
- **Arbitrary shapes**: Finds clusters of any shape, not just spherical
- **Hierarchical**: Builds a hierarchy of clusters at different scales

**The challenge**: Even 384 dimensions (embeddings) can suffer from the curse of dimensionality for density-based methods!

### Initial Attempt: HDBSCAN on Raw Embeddings

Let's first try HDBSCAN directly on our 384-dimensional embeddings to see what happens.

In [ ]:
# Apply HDBSCAN to raw embeddings
print("Applying HDBSCAN to raw embeddings (384 dimensions)...")

clusterer_hdbscan_initial = hdbscan.HDBSCAN(
    min_cluster_size=30,
    min_samples=5,
    metric='euclidean',
    cluster_selection_method='eom'
)

clusters_hdbscan_initial = clusterer_hdbscan_initial.fit_predict(embeddings)

# Count clusters
n_clusters_initial = len(set(clusters_hdbscan_initial)) - (1 if -1 in clusters_hdbscan_initial else 0)
n_noise_initial = (clusters_hdbscan_initial == -1).sum()

print(f"✓ HDBSCAN clustering complete")
print(f"  Clusters found: {n_clusters_initial}")
print(f"  Noise points: {n_noise_initial} ({100*n_noise_initial/len(df):.1f}%)")

if n_noise_initial / len(df) > 0.3:
    print(f"\n  ⚠️ WARNING: High percentage of noise!")
    print(f"  Even 384 dimensions can be too high for density-based clustering")

### The Curse of Dimensionality

Even though embeddings are much lower-dimensional than TF-IDF (384 vs 5000), they can still suffer from the curse of dimensionality for density-based methods.

**The problem**: In high-dimensional spaces, distances become less meaningful:
- All points appear roughly equidistant
- "Dense" regions are actually sparse
- HDBSCAN struggles to find meaningful clusters

**The solution**: **UMAP** (Uniform Manifold Approximation and Projection)
- Non-linear dimensionality reduction
- Preserves both local and global structure
- Designed specifically for clustering (unlike PCA which focuses on variance)
- Reduces to ~50 dimensions while maintaining semantic relationships

In [ ]:
# Demonstrate the curse of dimensionality with embeddings
print("Demonstrating curse of dimensionality in embedding space...")

# Sample documents for faster computation
sample_size = 500
sample_indices_emb = np.random.choice(len(embeddings), size=sample_size, replace=False)
embeddings_sample = embeddings[sample_indices_emb]

# Calculate pairwise distances
from sklearn.metrics.pairwise import euclidean_distances
distances_emb = euclidean_distances(embeddings_sample)

# Get distances to nearest neighbor
nearest_distances_emb = np.partition(distances_emb, 1, axis=1)[:, 1]

# Plot distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(nearest_distances_emb, bins=50, color='teal', edgecolor='black', alpha=0.7)
plt.xlabel('Distance to Nearest Neighbor', fontsize=12)
plt.ylabel('Number of Documents', fontsize=12)
plt.title(f'Embedding Space ({embeddings.shape[1]} dimensions)\nStill high-dimensional!', 
          fontsize=12, fontweight='bold')
plt.axvline(x=nearest_distances_emb.mean(), color='red', linestyle='--', linewidth=2,
            label=f'Mean = {nearest_distances_emb.mean():.2f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
all_distances_emb = distances_emb[np.triu_indices_from(distances_emb, k=1)]
plt.hist(all_distances_emb, bins=50, color='coral', edgecolor='black', alpha=0.7)
plt.xlabel('Pairwise Distance', fontsize=12)
plt.ylabel('Number of Pairs', fontsize=12)
plt.title('Distribution of All Pairwise Distances\nNarrow range = curse of dimensionality', 
          fontsize=12, fontweight='bold')
plt.axvline(x=all_distances_emb.mean(), color='red', linestyle='--', linewidth=2,
            label=f'Mean = {all_distances_emb.mean():.2f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

ratio_emb = nearest_distances_emb.std() / nearest_distances_emb.mean()
print(f"\n✓ OBSERVATION: In {embeddings.shape[1]} dimensions:")
print(f"  Distance variability (std/mean): {ratio_emb:.3f}")
print(f"  Still relatively low → distances are less meaningful because everything is about equally close")
print(f"  Solution: UMAP to reduce to ~50 dimensions")

### Solution: UMAP Dimensionality Reduction

We'll use **UMAP** to reduce from 384 dimensions to 50 dimensions.

**Why UMAP (not PCA)?**
- **Non-linear**: Captures complex relationships PCA misses
- **Preserves local structure**: Keeps similar documents close together
- **Preserves global structure**: Maintains overall data topology
- **Designed for clustering**: Works better with density-based methods

**Why 50 dimensions?**
- Balances information retention and curse of dimensionality
- Enough to preserve semantic structure
- Low enough for HDBSCAN to work well

In [ ]:
# Apply UMAP for dimensionality reduction
print("Applying UMAP dimensionality reduction...")
print(f"  Reducing from {embeddings.shape[1]} to 50 dimensions...")

# UMAP parameters
n_components_umap = 50
n_neighbors = 15  # Balance between local and global structure
min_dist = 0.1    # Minimum distance between points in low-dimensional space

umap_reducer = umap.UMAP(
    n_components=n_components_umap,
    n_neighbors=n_neighbors,
    min_dist=min_dist,
    metric='cosine',  # Good for embeddings
    random_state=42,
    verbose=True
)

# Fit and transform (this may take a minute)
embeddings_umap = umap_reducer.fit_transform(embeddings)

print(f"\n✓ UMAP reduction complete")
print(f"  Original shape: {embeddings.shape}")
print(f"  Reduced shape: {embeddings_umap.shape}")
print(f"  Compression ratio: {embeddings.shape[1] / embeddings_umap.shape[1]:.1f}x")

In [ ]:
# Verify that distances are more meaningful after UMAP
print("Comparing distances before and after UMAP...")

# Sample for faster computation
embeddings_umap_sample = embeddings_umap[sample_indices_emb]

# Calculate distances in UMAP space
distances_umap = euclidean_distances(embeddings_umap_sample)
nearest_distances_umap = np.partition(distances_umap, 1, axis=1)[:, 1]

# Plot comparison
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(nearest_distances_emb, bins=50, color='teal', edgecolor='black', alpha=0.7, label='Original (384D)')
plt.hist(nearest_distances_umap, bins=50, color='orange', edgecolor='black', alpha=0.7, label='After UMAP (50D)')
plt.xlabel('Distance to Nearest Neighbor', fontsize=12)
plt.ylabel('Number of Documents', fontsize=12)
plt.title('Nearest Neighbor Distances: Before vs After UMAP', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
# Compare ratios
ratio_umap = nearest_distances_umap.std() / nearest_distances_umap.mean()

plt.bar(['Original\n(384D)', f'UMAP\n({n_components_umap}D)'], 
        [ratio_emb, ratio_umap],
        color=['teal', 'orange'], edgecolor='black')
plt.ylabel('Std/Mean Ratio', fontsize=12)
plt.title('Distance Variability (Higher = Better Separation)', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ IMPROVEMENT:")
print(f"  Original space: std/mean = {ratio_emb:.3f}")
print(f"  UMAP space: std/mean = {ratio_umap:.3f}")
print(f"  Change: {((ratio_umap - ratio_emb) / ratio_emb * 100):+.1f}%")
print(f"\n  Higher ratio = better separation between near and far points")
print(f"  UMAP makes distances more meaningful for HDBSCAN!")

### Re-running HDBSCAN on UMAP-Reduced Embeddings

Now let's apply HDBSCAN to the UMAP-reduced embeddings and see if we get better results.

In [ ]:
# Apply HDBSCAN to UMAP-reduced embeddings
print("Applying HDBSCAN to UMAP-reduced embeddings...")

clusterer_hdbscan = hdbscan.HDBSCAN(
    min_cluster_size=30,
    min_samples=5,
    metric='euclidean',
    cluster_selection_method='eom'
)

clusters_hdbscan = clusterer_hdbscan.fit_predict(embeddings_umap)

print(f"✓ HDBSCAN clustering complete")

# Count clusters
n_clusters_found = len(set(clusters_hdbscan)) - (1 if -1 in clusters_hdbscan else 0)
n_noise = (clusters_hdbscan == -1).sum()

print(f"  Clusters found: {n_clusters_found}")
print(f"  Noise points: {n_noise} ({100*n_noise/len(df):.1f}%)")

# Compare with initial attempt
print(f"\n  Comparison with high-dimensional clustering:")
print(f"    Before UMAP: {n_clusters_initial} clusters, {100*n_noise_initial/len(df):.1f}% noise")
print(f"    After UMAP:  {n_clusters_found} clusters, {100*n_noise/len(df):.1f}% noise")
print(f"    Improvement: {n_noise_initial - n_noise} fewer noise points!")

# Add to dataframe
df['cluster_hdbscan'] = clusters_hdbscan

# Show cluster sizes
print(f"\n  Cluster sizes:")
for i in sorted(set(clusters_hdbscan)):
    if i == -1:
        print(f"    Noise: {(clusters_hdbscan == i).sum()} documents ({100*(clusters_hdbscan == i).sum()/len(df):.1f}%)")
    else:
        count = (clusters_hdbscan == i).sum()
        print(f"    Cluster {i}: {count} documents ({100*count/len(df):.1f}%)")

if n_noise / len(df) < 0.3:
    print(f"\n  ✅ SUCCESS: Noise reduced to acceptable level!")
else:
    print(f"\n  ⚠️ Still high noise - may need to adjust parameters")

### Diagnostic 1: Top Words Per Cluster (HDBSCAN)

Let's see what topics HDBSCAN discovered without being told how many to find.

In [ ]:
# Get top words for HDBSCAN clusters (excluding noise)
valid_clusters = [c for c in set(clusters_hdbscan) if c != -1]
top_words_hdbscan = {}

for cluster_id in valid_clusters:
    cluster_mask = clusters_hdbscan == cluster_id
    cluster_docs = X_tfidf[cluster_mask]
    
    # Average TF-IDF scores across cluster
    avg_scores = np.asarray(cluster_docs.mean(axis=0)).flatten()
    
    # Get top words
    feature_names = vectorizer_tfidf.get_feature_names_out()
    top_indices = avg_scores.argsort()[-10:][::-1]
    top_words_hdbscan[cluster_id] = [(feature_names[i], avg_scores[i]) for i in top_indices]

# Display
print("=" * 80)
print(f"TOP 10 WORDS PER CLUSTER (HDBSCAN - {n_clusters_found} clusters discovered)")
print("=" * 80)

for cluster_id in sorted(valid_clusters):
    words = top_words_hdbscan[cluster_id]
    print(f"\nCluster {cluster_id}:")
    word_list = [f"{word} ({score:.3f})" for word, score in words]
    print(f"  {', '.join(word_list)}")

print("\n" + "=" * 80)
print(f"✓ OBSERVATION: HDBSCAN found {n_clusters_found} clusters (not 5!)")
print("This suggests topics might be more granular than news categories.")
print("=" * 80)

In [ ]:
# Visualize top words for each HDBSCAN cluster
n_cols = 3
n_rows = (n_clusters_found + n_cols - 1) // n_cols  # Ceiling division

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5*n_rows))
if n_clusters_found > 1:
    axes = axes.flatten()
else:
    axes = [axes]

for idx, cluster_id in enumerate(sorted(valid_clusters)):
    words, scores = zip(*top_words_hdbscan[cluster_id])
    
    axes[idx].barh(range(len(words)), scores, color='coral')
    axes[idx].set_yticks(range(len(words)))
    axes[idx].set_yticklabels(words)
    axes[idx].set_xlabel('Average TF-IDF Score', fontsize=10)
    axes[idx].set_title(f'Cluster {cluster_id} Top Words', fontsize=11, fontweight='bold')
    axes[idx].invert_yaxis()
    axes[idx].grid(axis='x', alpha=0.3)

# Remove extra subplots
for idx in range(n_clusters_found, len(axes)):
    fig.delaxes(axes[idx])

plt.suptitle(f'Top Words Per Cluster (HDBSCAN) - {n_clusters_found} Clusters Discovered', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### Diagnostic 2: Cluster Quality Metrics (HDBSCAN)

In [ ]:
# Calculate cluster quality metrics for HDBSCAN
print("Calculating cluster quality metrics for HDBSCAN...")

# For metrics, we'll exclude noise points (-1)
non_noise_mask = clusters_hdbscan != -1
clusters_hdbscan_no_noise = clusters_hdbscan[non_noise_mask]
categories_no_noise = df['category'][non_noise_mask]

# Cluster balance (excluding noise)
cluster_sizes_hdbscan = np.bincount(clusters_hdbscan_no_noise)
cluster_balance_hdbscan = entropy(cluster_sizes_hdbscan / cluster_sizes_hdbscan.sum()) / np.log(len(cluster_sizes_hdbscan))

# Cluster purity (excluding noise)
purity_hdbscan = cluster_purity(categories_no_noise, clusters_hdbscan_no_noise)

# Top word distinctiveness
word_overlap_hdbscan = top_word_overlap(top_words_hdbscan)

print(f"✓ Cluster quality analysis complete")
print(f"\n  Clusters found: {n_clusters_found}")
print(f"  Noise points: {n_noise} ({100*n_noise/len(df):.1f}%)")
print(f"  Cluster balance: {cluster_balance_hdbscan:.3f}")
print(f"  Cluster purity: {purity_hdbscan:.3f}")
print(f"  Top word overlap: {word_overlap_hdbscan:.3f}")
print(f"\n  Comparison with k=5 methods:")
print(f"    BoW:        balance={cluster_balance:.3f}, purity={purity_bow:.3f}")
print(f"    TF-IDF:     balance={cluster_balance_tfidf:.3f}, purity={purity_tfidf:.3f}")
print(f"    LDA:        balance={cluster_balance_lda:.3f}, purity={purity_lda:.3f}")
print(f"    Embeddings: balance={cluster_balance_emb:.3f}, purity={purity_emb:.3f}")
print(f"    HDBSCAN:    balance={cluster_balance_hdbscan:.3f}, purity={purity_hdbscan:.3f}")

In [ ]:
# Visualize cluster sizes and composition
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Cluster size distribution
cluster_sizes_hdbscan_pct = cluster_sizes_hdbscan / cluster_sizes_hdbscan.sum() * 100
axes[0].bar(range(len(cluster_sizes_hdbscan)), cluster_sizes_hdbscan_pct, color='coral', edgecolor='black')
axes[0].set_xlabel('Cluster ID', fontsize=12)
axes[0].set_ylabel('Percentage of Documents (excl. noise)', fontsize=12)
axes[0].set_title(f'HDBSCAN: {n_clusters_found} Clusters (Balance = {cluster_balance_hdbscan:.3f})', 
                  fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Right: Cluster composition by category (excluding noise)
contingency_hdbscan = pd.crosstab(clusters_hdbscan_no_noise, categories_no_noise, normalize='index') * 100
contingency_hdbscan.plot(kind='bar', stacked=True, ax=axes[1], colormap='Set3')
axes[1].set_xlabel('Cluster ID', fontsize=12)
axes[1].set_ylabel('Percentage', fontsize=12)
axes[1].set_title(f'HDBSCAN: Composition (Purity = {purity_hdbscan:.3f})', 
                  fontsize=12, fontweight='bold')
axes[1].legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ OBSERVATION: HDBSCAN found {n_clusters_found} clusters instead of 5")
print("This suggests the data has more natural groupings than the 5 predefined categories!")
print("Purity and cluster balance have declined - primarily because business and politics have become difficult to separate.")

### Diagnostic 3: Analyzing Noise Points

One unique feature of HDBSCAN is that it can identify outliers - documents that don't fit well into any cluster.

In [ ]:
# Analyze noise points
if n_noise > 0:
    print("=" * 80)
    print("NOISE POINT ANALYSIS")
    print("=" * 80)
    
    noise_mask = clusters_hdbscan == -1
    noise_docs = df[noise_mask]
    
    # Category distribution of noise
    print(f"\nNoise points by category:")
    noise_categories = noise_docs['category'].value_counts()
    for cat, count in noise_categories.items():
        print(f"  {cat}: {count} ({100*count/n_noise:.1f}% of noise)")
    
    # Show a few examples
    print(f"\nExample noise documents (don't fit well into any cluster):")
    for i, idx in enumerate(noise_docs.index[10:13]):
        print(f"\n{'-'*80}")
        print(f"Example {i+1}: Category = {df.iloc[idx]['category']}")
        print(f"{df.iloc[idx]['text'][:800]}...")
    
    print(f"\n{'='*80}")
    print("✓ INSIGHT: Noise points might be:")
    print("  - Documents that span multiple topics")
    print("  - Unusual or unique documents")
    print("  - Documents that don't fit the main themes")
else:
    print("No noise points detected by HDBSCAN")

### Diagnostic 4: Cluster Hierarchy and Confidence

HDBSCAN builds a hierarchy of clusters and provides confidence scores for each assignment.

In [ ]:
# Visualize cluster hierarchy
print("Visualizing cluster hierarchy...")

# Get cluster probabilities (how confident is HDBSCAN about each assignment?)
cluster_probs = clusterer_hdbscan.probabilities_

# Show distribution of cluster membership probabilities
plt.figure(figsize=(12, 5))

plt.hist(cluster_probs[non_noise_mask], bins=30, color='coral', edgecolor='black', alpha=0.7)
plt.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Low confidence threshold')
plt.xlabel('Cluster Membership Probability', fontsize=12)
plt.ylabel('Number of Documents', fontsize=12)
plt.title('HDBSCAN: Cluster Membership Confidence\n(Higher = More Confident Assignment)', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

low_confidence = (cluster_probs < 0.5).sum()
print(f"\nDocuments with low confidence (< 0.5): {low_confidence} ({100*low_confidence/len(df):.1f}%)")
print("These documents are on the boundary between clusters")

### Diagnostic 5: Visualizing Clusters in 2D

Let's visualize the HDBSCAN clusters in 2D space using UMAP.

In [ ]:
# Reduce UMAP embeddings to 2D for visualization
print("Reducing to 2D for visualization...")

umap_2d = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

embeddings_2d_umap = umap_2d.fit_transform(embeddings)

print("✓ 2D UMAP complete")

# Plot HDBSCAN clusters
plt.figure(figsize=(14, 8))

# Plot each cluster with different color
hdbscan_colors = plt.cm.tab20(np.linspace(0, 1, n_clusters_found + 1))

for idx, cluster_id in enumerate(sorted(valid_clusters)):
    mask = clusters_hdbscan == cluster_id
    plt.scatter(embeddings_2d_umap[mask, 0], embeddings_2d_umap[mask, 1],
                c=[hdbscan_colors[idx]], label=f'Cluster {cluster_id}',
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

# Plot noise points
if n_noise > 0:
    noise_mask = clusters_hdbscan == -1
    plt.scatter(embeddings_2d_umap[noise_mask, 0], embeddings_2d_umap[noise_mask, 1],
                c='lightgray', label='Noise', marker='x',
                alpha=0.5, s=30)

plt.xlabel('UMAP Dimension 1', fontsize=12)
plt.ylabel('UMAP Dimension 2', fontsize=12)
plt.title(f'HDBSCAN Clusters in 2D (Embeddings + UMAP)\n{n_clusters_found} Clusters Discovered', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=9, loc='best', ncol=2)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✓ OBSERVATION: HDBSCAN adapts to data density")
print(f"It found {n_clusters_found} clusters instead of forcing k=5")
print("Note that a small 'bridge' of documents seem to connect the huge 'business' and 'politics' clusters.")

### Reflection: What Did We Learn?

**Key insights from HDBSCAN + Embeddings + UMAP**:

1. ✅ **No need to specify k**: Found {n_clusters_found} clusters automatically
2. ✅ **Semantic understanding**: Embeddings capture meaning
3. ✅ **UMAP helps**: Reduces dimensionality while preserving structure
4. ✅ **Identifies outliers**: {n_noise} documents labeled as noise
5. ✅ **Confidence scores**: Shows which assignments are uncertain
6. ✅ **Adapts to density**: Clusters can have different sizes naturally

**The complete pipeline**:
1. **Text → Embeddings**: Capture semantic meaning (384D)
2. **Embeddings → UMAP**: Reduce dimensionality (50D)
3. **UMAP → HDBSCAN**: Discover clusters automatically

**When to use this approach**:
- ✅ Exploratory analysis (don't know how many topics)
- ✅ Semantic similarity is important
- ✅ Uneven topic distributions
- ✅ Want to identify outliers
- ✅ Have computational resources

**Trade-offs**:
- ⚠️ **Computational cost**: Embeddings + UMAP + HDBSCAN takes time
- ⚠️ **Complexity**: More moving parts than simple K-Means
- ⚠️ **Interpretability**: Harder to explain than word counts
- ⚠️ **Parameter sensitivity**: HDBSCAN results depend on min_cluster_size, min_samples

**The big question**: Are news categories the same as topics?
- HDBSCAN suggests: **No!** The data contains multiple separate dense regions.
- This could mean topics are more granular, or categories overlap
- Always question your assumptions about k!

### Final Comparison: All Methods

Let's compare all the methods we've explored in this exercise.

In [ ]:
# Create comprehensive comparison table
comparison_data = {
    'Method': [
        'BoW + K-Means',
        'TF-IDF + K-Means',
        'BoW + LDA',
        'Embeddings + K-Means',
        'Embeddings + UMAP + HDBSCAN'
    ],
    'Representation': [
        'Lexical (sparse)',
        'Lexical (sparse)',
        'Lexical (sparse)',
        'Semantic (dense)',
        'Semantic (dense)'
    ],
    'Dimensions': [
        X_bow.shape[1],
        X_tfidf.shape[1],
        X_lda.shape[1],
        embeddings.shape[1],
        embeddings_umap.shape[1]
    ],
    'Algorithm': [
        'K-Means',
        'K-Means',
        'LDA',
        'K-Means',
        'HDBSCAN'
    ],
    'Clusters': [
        n_clusters,
        n_clusters,
        n_topics,
        n_clusters,
        n_clusters_found
    ],
    'Balance': [
        f'{cluster_balance:.3f}',
        f'{cluster_balance_tfidf:.3f}',
        f'{cluster_balance_lda:.3f}',
        f'{cluster_balance_emb:.3f}',
        f'{cluster_balance_hdbscan:.3f}'
    ],
    'Purity': [
        f'{purity_bow:.3f}',
        f'{purity_tfidf:.3f}',
        f'{purity_lda:.3f}',
        f'{purity_emb:.3f}',
        f'{purity_hdbscan:.3f}'
    ],
    'Word Overlap': [
        f'{word_overlap_bow:.3f}',
        f'{word_overlap_tfidf:.3f}',
        f'{word_overlap_lda:.3f}',
        f'{word_overlap_emb:.3f}',
        f'{word_overlap_hdbscan:.3f}'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("=" * 100)
print("COMPREHENSIVE METHOD COMPARISON")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)
print("\n✓ Key Takeaways:")
print("  1. TF-IDF improves over BoW by downweighting common words")
print("  2. LDA provides soft clustering (documents can have multiple topics)")
print("  3. Embeddings capture semantic meaning (best purity)")
print("  4. HDBSCAN discovers natural number of clusters")
print("  5. UMAP helps HDBSCAN work in high-dimensional spaces")
print("\n💡 Recommendation: Start simple (TF-IDF + K-Means), then try embeddings if needed")
print("=" * 100)

### Further Exploration

If you want to go deeper:

1. **Try different datasets**: Load your own data or try other public datasets
2. **Experiment with parameters**: 
   - Different numbers of topics/clusters
   - Different embedding models (all-mpnet-base-v2 for better quality)
   - Different UMAP parameters (n_neighbors, min_dist)
   - Different HDBSCAN parameters (min_cluster_size, min_samples)

3. **Combine methods**: 
   - Use LDA to discover topics, then embeddings to find similar documents
   - Use HDBSCAN to find number of clusters, then K-Means with that k
   - Use embeddings for similarity, TF-IDF for interpretation

4. **Advanced techniques**:
   - **BERTopic**: Combines embeddings, UMAP, and HDBSCAN automatically
   - **Top2Vec**: Similar to BERTopic but with different approach
   - **Fine-tuning**: Train embeddings on your specific domain
   - **Hierarchical clustering**: Explore topic hierarchies

5. **Temporal analysis**: If your data has timestamps, track topic evolution over time

6. **Multilingual analysis**: Modern embedding models work across languages!

7. **Interactive exploration**: Use tools like Jupyter widgets to explore clusters interactively

**Resources**:
- **Sentence Transformers**: https://www.sbert.net/
- **UMAP**: https://umap-learn.readthedocs.io/
- **HDBSCAN**: https://hdbscan.readthedocs.io/
- **BERTopic**: https://maartengr.github.io/BERTopic/
- **Scikit-learn**: https://scikit-learn.org/stable/modules/clustering.html
- **Hugging Face**: https://huggingface.co/models (for more embedding models)

**Papers**:
- Blei et al. (2003): "Latent Dirichlet Allocation" (LDA)
- McInnes et al. (2018): "UMAP: Uniform Manifold Approximation and Projection"
- McInnes et al. (2017): "hdbscan: Hierarchical density based clustering"
- Reimers & Gurevych (2019): "Sentence-BERT" (sentence embeddings)